In [ ]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from numpy import inner
import plotly.express as px
from google.colab import drive
from haversine import haversine
import plotly.graph_objects as go

from folium.vector_layers import Popup
# localização central para cada tipo de tráfego
import folium
#!pip install haversine
drive.mount('/content/drive')
df_raw = pd.read_csv('/content/drive/MyDrive/FTC/train.csv')
df_raw.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [ ]:
df1 = df_raw.copy()

## Converting Data Type

In [ ]:
# convering Delivery_person_Age
line_nan = (df1['Delivery_person_Age'] != 'NaN ') 
df1 = df1.loc[line_nan, :].copy()


line_nan = (df1['Road_traffic_density'] != 'NaN ') 
df1 = df1.loc[line_nan, :].copy()

line_nan = (df1['City'] != 'NaN ') 
df1 = df1.loc[line_nan, :].copy()

line_nan = (df1['Festival'] != 'NaN ') 
df1 = df1.loc[line_nan, :].copy()
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].astype(int)

# convering Delivery_person_Ratings
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype( float )

# converting Order_Date 
df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], format = '%d-%m-%Y')

# convertig multiple_deliveries
aux = (df1['multiple_deliveries']!= 'NaN ')
df1 = df1.loc[aux, :].copy()

df1['multiple_deliveries'] = df1['multiple_deliveries'].astype( int )

# removing spaces in strings or objects
#df1 = df1.reset_index(drop = True)
#for i in range(len(df1)):
#   df1.loc[i,'ID'] = df1.loc[i, 'ID'].strip()

df1.loc[:,'ID']=df1.loc[:,'ID'].str.strip()
df1.loc[:,'Delivery_person_ID']=df1.loc[:,'Delivery_person_ID'].str.strip()
df1.loc[:,'Type_of_order']=df1.loc[:,'Type_of_order'].str.strip()
df1.loc[:,'Road_traffic_density']=df1.loc[:,'Road_traffic_density'].str.strip()
df1.loc[:,'Type_of_vehicle']=df1.loc[:,'Type_of_vehicle'].str.strip()
df1.loc[:,'Festival']=df1.loc[:,'Festival'].str.strip()
df1.loc[:,'City']=df1.loc[:,'City'].str.strip()

df1['Time_taken(min)'] = df1['Time_taken(min)'].apply( lambda x: x.split( '(min)')[1])
df1['Time_taken(min)'] = df1['Time_taken(min)'].astype( int )

In [ ]:
df1.iloc[0,:]

ID                                          0x4607
Delivery_person_ID                  INDORES13DEL02
Delivery_person_Age                             37
Delivery_person_Ratings                        4.9
Restaurant_latitude                      22.745049
Restaurant_longitude                     75.892471
Delivery_location_latitude               22.765049
Delivery_location_longitude              75.912471
Order_Date                     2022-03-19 00:00:00
Time_Orderd                               11:30:00
Time_Order_picked                         11:45:00
Weatherconditions                 conditions Sunny
Road_traffic_density                          High
Vehicle_condition                                2
Type_of_order                                Snack
Type_of_vehicle                         motorcycle
multiple_deliveries                              0
Festival                                        No
City                                         Urban
Time_taken(min)                

## Quantidades de pedidos por dia

In [ ]:
cols = ['ID', 'Order_Date']
df_aux = df1.loc[:, cols].groupby(['Order_Date']).count().reset_index()

px.bar(df_aux, x ='Order_Date', y = 'ID')

## Quantidade de pedidos por semana

In [ ]:
# treanformando em semanas considerando primeir odia util seguna usar W se for Domingo usar U
df1['week_of_year'] = df1['Order_Date'].dt.strftime('%U')
df_aux = df1.loc[:, ['ID', 'week_of_year']].groupby(['week_of_year']).count().reset_index()

px.line(df_aux, x = 'week_of_year', y = 'ID')

## Distribuicao de pedidos por tipo de trafego

In [ ]:
df_aux = df1.loc[:, ['ID', 'Road_traffic_density']].groupby(['Road_traffic_density']).count().reset_index()
df_aux = df_aux.loc[df_aux['Road_traffic_density']!= 'NaN', :]
df_aux['%_delivery'] = df_aux['ID'] / df_aux['ID'].sum()

#df_aux
px.pie(df_aux, values = '%_delivery', names = 'Road_traffic_density')


## Volume de pedidos por cidade e trafego

In [ ]:
df_aux = df1.loc[:, ['ID', 'City', 'Road_traffic_density']].groupby(['City','Road_traffic_density']).count().reset_index()
df_aux = df_aux.loc[df_aux['City'] != 'NaN', :]
df_aux = df_aux.loc[df_aux['Road_traffic_density'] != 'NaN', :]

#df_aux
px.scatter(df_aux, x = 'City', y = 'Road_traffic_density', size = 'ID', color = 'City')

## quantidade de pedidos por entregador por semana

In [ ]:

# quantidad de pedidos por semana/numero unico de entregadores por semana
df_aux1 = df1.loc[:, ['ID', 'week_of_year']].groupby(['week_of_year']).count().reset_index()
df_aux2 = df1.loc[:, ['Delivery_person_ID', 'week_of_year']].groupby('week_of_year').nunique().reset_index()

df_aux = pd.merge(df_aux1,df_aux2, how = 'inner' )
df_aux['order_by_delivey'] = df_aux['ID'] / df_aux['Delivery_person_ID']
#df_aux.head()

px.line(df_aux, x = 'week_of_year', y = 'order_by_delivey')

In [ ]:
df_aux = df1.loc[:, ['City', 'Road_traffic_density', 'Delivery_location_latitude','Delivery_location_longitude']].groupby(['City', 'Road_traffic_density']).median().reset_index()
df_aux = df_aux.loc[df_aux['City'] != 'NaN', :]
df_aux = df_aux.loc[df_aux['Road_traffic_density'] != 'NaN', :]


map = folium.Map()

for index, location_info in df_aux.iterrows():
  folium.Marker([location_info['Delivery_location_latitude'],
                 location_info['Delivery_location_longitude']],
                 popup= location_info[['City', 'Road_traffic_density']]).add_to(map)
  

print(map)

# Delivery vision




### A menor e maior idade dos entregadores



In [ ]:
print('A maior idade dos entregadores e: ', df1.loc[:, 'Delivery_person_Age'].max())
print('A maior idade dos entregadores e: ', df1.loc[:, 'Delivery_person_Age'].min())

A maior idade dos entregadores e:  39
A maior idade dos entregadores e:  20


### Pior e melhor condicao de veiculo

In [ ]:
print('A pior condicao do veiculo: ', df1.loc[:, 'Vehicle_condition'].max())
print('A pior condicao do veiculo: ', df1.loc[:, 'Vehicle_condition'].min())

A pior condicao do veiculo:  2
A pior condicao do veiculo:  0


### A avaliacao media por entregador

In [ ]:
df_average_rating_per_deliver = df1.loc[:, ['Delivery_person_ID', 'Delivery_person_Ratings']].groupby(['Delivery_person_ID']).mean().reset_index()
df_average_rating_per_deliver.head()

,Delivery_person_ID,Delivery_person_Ratings
0,AGRRES010DEL01,4.761538
1,AGRRES010DEL02,4.671429
2,AGRRES010DEL03,4.575000
3,AGRRES01DEL01,4.522222
4,AGRRES01DEL02,4.700000


### A Avaliaca ,edia e o desvio padrao por tipo de trafego

In [ ]:
#using one line
#df_average_rating_by_trafic = df1.loc[:, ['Delivery_person_Ratings','Road_traffic_density' ]].groupby(['Road_traffic_density']).agg({'Delivery_person_Ratings':['mean','std']})

#using two lines
df_avg_std_by_trafic = (df1.loc[:, ['Delivery_person_Ratings','Road_traffic_density' ]]
                              .groupby(['Road_traffic_density'])
                              .agg({'Delivery_person_Ratings':['mean','std']}))

#df_std_rating_by_trafic = df1.loc[:, ['Delivery_person_Ratings','Road_traffic_density' ]].groupby(['Road_traffic_density']).std().reset_index()

# Manipulation dataframe column name
df_avg_std_by_trafic.columns = ['delivery_mean', 'delivery_std']
# Manipulation dataframe format

df_avg_std_by_trafic.reset_index()

,Road_traffic_density,delivery_mean,delivery_std
0,High,4.652230,0.273044
1,Jam,4.594019,0.329778
2,Low,4.645011,0.338080
3,Medium,4.660138,0.274245


### A Avaliacao de media e desvio padrao por condicoes climaticas

In [ ]:
#using one line
#df_average_rating_rating_by_wather = df1.loc[:, ['Delivery_person_Ratings','Weatherconditions' ]].groupby(['Weatherconditions']).agg({'Delivery_person_Ratings':['mean','std']})

#using two lines
df_avg_std_rating_by_wather = (df1.loc[:, ['Delivery_person_Ratings','Weatherconditions' ]]
                              .groupby(['Weatherconditions'])
                              .agg({'Delivery_person_Ratings':['mean','std']}))

#df_std_rating_rating_by_wather = df1.loc[:, ['Delivery_person_Ratings','Weatherconditions' ]].groupby(['Weatherconditions']).std().reset_index()

# Manipulation dataframe column name
df_avg_std_rating_by_wather.columns = ['delivery_mean', 'delivery_std']
# Manipulation dataframe format

df_avg_std_rating_by_wather.reset_index()

,Weatherconditions,delivery_mean,delivery_std
0,conditions Cloudy,4.651871,0.281197
1,conditions Fog,4.652965,0.275060
2,conditions Sandstorms,4.611748,0.310852
3,conditions Stormy,4.611819,0.313096
4,conditions Sunny,4.654868,0.396674
5,conditions Windy,4.616128,0.304565


### 10 entregadores mais rapidos da cidade



In [ ]:
df2 = df1.loc[:, ['Delivery_person_ID', 'Time_taken(min)', 'City']].groupby(['City', 'Delivery_person_ID']).median().sort_values(['Time_taken(min)', 'City'], ascending = False).reset_index()

df_aux01 = df2.loc[df2['City'] == 'Metropolitian', : ].head(10)
df_aux02 = df2.loc[df2['City'] == 'Urban', : ].head(10)
df_aux03 = df2.loc[df2['City'] == 'Semi-Urban', : ].head(10)

pd.concat([df_aux01,df_aux02,df_aux03]).reset_index(drop = True)


,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,AURGRES11DEL03,41.5
1,Metropolitian,AGRRES02DEL01,40.0
2,Metropolitian,ALHRES18DEL02,39.0
3,Metropolitian,KOLRES03DEL03,39.0
4,Metropolitian,LUDHRES14DEL01,39.0
5,Metropolitian,LUDHRES17DEL03,39.0
6,Metropolitian,ALHRES03DEL01,38.5
7,Metropolitian,KOCRES20DEL01,38.5
8,Metropolitian,ALHRES12DEL01,37.5
9,Metropolitian,BHPRES19DEL01,37.0


### 10 entregadores mais lentos da cidade



In [ ]:
df2 = df1.loc[:, ['Delivery_person_ID', 'Time_taken(min)', 'City']].groupby(['City', 'Delivery_person_ID']).mean().sort_values(['Time_taken(min)', 'City'], ascending = True).reset_index()

df_aux01 = df2.loc[df2['City'] == 'Metropolitian', : ].head(10)
df_aux02 = df2.loc[df2['City'] == 'Urban', : ].head(10)
df_aux03 = df2.loc[df2['City'] == 'Semi-Urban', : ].head(10)

pd.concat([df_aux01,df_aux02,df_aux03]).reset_index(drop = True)


,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,KNPRES03DEL02,15.750000
1,Metropolitian,ALHRES02DEL02,17.800000
2,Metropolitian,KNPRES01DEL01,19.125000
3,Metropolitian,KOLRES01DEL03,19.125000
4,Metropolitian,KOCRES02DEL02,19.250000
5,Metropolitian,KOCRES08DEL03,19.800000
6,Metropolitian,KOCRES16DEL03,20.000000
7,Metropolitian,KOCRES02DEL03,20.375000
8,Metropolitian,GOARES14DEL01,20.400000
9,Metropolitian,DEHRES20DEL03,20.428571


# Restaurante

In [ ]:
delivery_unique = len(df1.loc[:, 'Delivery_person_ID'].unique())

In [ ]:
delivery_unique

1320

## Distancia media de restaurantes e dos locais de entrega

In [ ]:
cols = ['Delivery_location_latitude','Delivery_location_longitude','Restaurant_latitude', 'Restaurant_longitude']


df1['distance'] = df1.loc[:, cols].apply( lambda x:
                              haversine(
                                (x['Restaurant_latitude'], x['Restaurant_longitude']),
                                (x['Delivery_location_latitude'], x['Delivery_location_longitude']) ), axis = 1)

avg_distance = df1['distance'].mean()

print('A distancia media e de: {:.2f}KM'.format(avg_distance) )

A distancia media e de: 27.44KM


In [ ]:
cols = ['Delivery_location_latitude', 'Delivery_location_longitude', 'Restaurant_latitude', 'Restaurant_longitude']
df1['distance'] = df1.loc[:, cols].apply(lambda x:
                                         haversine( (x['Restaurant_latitude'], x['Restaurant_longitude']),
                                                    (x['Delivery_location_latitude'], x['Delivery_location_longitude'])), axis = 1)

avg_distance = df1.loc[:, ['City', 'distance']].groupby('City').mean().reset_index()


fig = go.Figure( data = [go.Pie(labels = avg_distance['City'], values = avg_distance['distance'], pull =[0,0.1,0])])
fig.show()


## tempo medio e desvio padrao  de entrega por cidade

In [ ]:
cols = ['Time_taken(min)', 'City']

df_aux = df1.loc[:, cols].groupby('City').agg( {'Time_taken(min)': ['mean', 'std']})

df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()
df_aux

,City,avg_time,std_time
0,Metropolitian,27.428083,9.133374
1,Semi-Urban,49.710526,2.724992
2,Urban,23.209379,8.858049


## tempo medio e desvio medio por entregar por cidade e tipo de pedido

In [ ]:
cols = ['Time_taken(min)', 'City', 'Type_of_order']

df_aux = df1.loc[:, cols].groupby(['City', 'Type_of_order']).agg( {'Time_taken(min)': ['mean', 'std']})

df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()
df_aux

,City,Type_of_order,avg_time,std_time
0,Metropolitian,Buffet,27.299008,9.153107
1,Metropolitian,Drinks,27.322691,9.041655
2,Metropolitian,Meal,27.616383,9.214536
3,Metropolitian,Snack,27.468414,9.119676
4,Semi-Urban,Buffet,49.707317,2.731702
5,Semi-Urban,Drinks,49.625000,2.459347
6,Semi-Urban,Meal,50.300000,3.041665
7,Semi-Urban,Snack,49.408163,2.707385
8,Urban,Buffet,23.560652,9.056348
9,Urban,Drinks,23.311977,8.927314


## O tempo medio e desvio padrao de entrega por cidade e tipo de trafego

In [ ]:
cols = ['Time_taken(min)', 'City', 'Road_traffic_density']

df_aux = df1.loc[:, cols].groupby(['City', 'Road_traffic_density']).agg( {'Time_taken(min)': ['mean', 'std']})

df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()
df_aux

,City,Road_traffic_density,avg_time,std_time
0,Metropolitian,High,28.140898,7.904645
1,Metropolitian,Jam,31.976991,9.476203
2,Metropolitian,Low,22.257675,6.794772
3,Metropolitian,Medium,27.729966,8.308064
4,Semi-Urban,High,50.125000,2.629956
5,Semi-Urban,Jam,49.841270,2.717095
6,Semi-Urban,Medium,47.400000,2.011080
7,Urban,High,24.305335,8.494842
8,Urban,Jam,27.993164,10.078271
9,Urban,Low,19.446809,6.319963


## Tempo medio de entrega durante os Festivais

In [ ]:
cols = ['Time_taken(min)', 'Festival']

df_aux = df1.loc[:, cols].groupby(['Festival']).agg( {'Time_taken(min)': ['mean', 'std']})

df_aux.columns = ['avg_time', 'std_time']
df_aux = df_aux.reset_index()
select_lines = df_aux['Festival'] == 'Yes'
df_aux = df_aux.loc[select_lines, :]
df_aux

,Festival,avg_time,std_time
1,Yes,45.518607,4.005399


## Unique deliverys